In [ ]:
import pandas as pd
from django.apps import apps
from apis_core.apis_vocabularies.models import PersonPlaceRelation

In [ ]:
models = list(apps.all_models["apis_vocabularies"].values())
models = [x for x in models if x.__name__.endswith("Relation")]

In [ ]:
values_list = ["id", "name"]
for x in models:
    model_name = x.__name__
    relation_class_name = model_name.lower().replace("relation", "")
    relation_class = apps.get_model(app_label="apis_relations", model_name=relation_class_name)
    items = x.objects.values_list(*values_list)
    df = pd.DataFrame(items, columns=values_list)
    for g, ndf in df.groupby("name"):
        if len(ndf) > 1:
            print(g, len(ndf))
            vocabs = list(ndf["id"])
            vocab_keep = x.objects.get(id=vocabs[0])
            relation_objects_keep = relation_class.objects.filter(relation_type=vocabs[0])
            relation_objects_change = relation_class.objects.filter(relation_type__in=vocabs[1:])
            print(f"keep: {relation_objects_keep.count()}; change: {relation_objects_change.count()}")
            for y in relation_objects_change:
                y.relation_type = vocab_keep
                y.save()
            for to_go in x.objects.filter(id__in=vocabs[1:]):
                to_go.delete()
    
